In [1]:
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
import torch.optim as optim
from sklearn.datasets import make_classification
from sklearn.model_selection import KFold
from sklearn.preprocessing import StandardScaler

# Load dataset
data = pd.read_csv('pseudodata_præoperation.csv')

#make pd to np
X = data.to_numpy()
y = np.random.choice([0, 1], size=len(data))

# Standardize the features
scaler = StandardScaler()
X = scaler.fit_transform(X)

# Convert to PyTorch tensors
X = torch.tensor(X, dtype=torch.float32)
y = torch.tensor(y, dtype=torch.float32).unsqueeze(1)

In [2]:
# Define the Artificial Neural Network Model
class ANNModel(nn.Module):
    def __init__(self, input_dim):
        super(ANNModel, self).__init__()
        self.fc1 = nn.Linear(input_dim, 10)
        self.relu = nn.ReLU()
        self.fc2 = nn.Linear(10, 1)
    
    def forward(self, x):
        out = self.fc1(x)
        out = self.relu(out)
        out = self.fc2(out)
        return torch.sigmoid(out)

input_dim = X.shape[1]

# Function to train and evaluate the model
def train_and_evaluate_model(X_train, y_train, X_val, y_val, input_dim, num_epochs=100, lr=0.01):
    model = ANNModel(input_dim)
    criterion = nn.BCELoss()
    optimizer = optim.SGD(model.parameters(), lr=lr)
    
    # Training loop
    for epoch in range(num_epochs):
        # Forward pass
        outputs = model(X_train)
        loss = criterion(outputs, y_train)
        
        # Backward pass and optimization
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
    
    # Evaluate on validation set
    with torch.no_grad():
        predicted = model(X_val).round()
        accuracy = (predicted.eq(y_val).sum() / float(y_val.shape[0])).item()
    
    return accuracy

# Outer 10-Fold Cross-Validation
outer_kf = KFold(n_splits=10, shuffle=True, random_state=42)
outer_accuracies = []

for outer_train_index, outer_test_index in outer_kf.split(X):
    X_outer_train, X_outer_test = X[outer_train_index], X[outer_test_index]
    y_outer_train, y_outer_test = y[outer_train_index], y[outer_test_index]

    # Inner 10-Fold Cross-Validation
    inner_kf = KFold(n_splits=10, shuffle=True, random_state=42)
    inner_accuracies = []

    for inner_train_index, inner_val_index in inner_kf.split(X_outer_train):
        X_inner_train, X_inner_val = X_outer_train[inner_train_index], X_outer_train[inner_val_index]
        y_inner_train, y_inner_val = y_outer_train[inner_train_index], y_outer_train[inner_val_index]

        accuracy = train_and_evaluate_model(X_inner_train, y_inner_train, X_inner_val, y_inner_val, input_dim)
        inner_accuracies.append(accuracy)
        print(f'Inner Fold Accuracy: {accuracy * 100:.2f}%')

    mean_inner_accuracy = sum(inner_accuracies) / len(inner_accuracies)
    print(f'Outer Fold Mean Inner Accuracy: {mean_inner_accuracy * 100:.2f}%')

    outer_accuracy = train_and_evaluate_model(X_outer_train, y_outer_train, X_outer_test, y_outer_test, input_dim)
    outer_accuracies.append(outer_accuracy)
    print(f'Outer Fold Accuracy: {outer_accuracy * 100:.2f}%')

print(f'Mean Outer Accuracy: {sum(outer_accuracies)/len(outer_accuracies) * 100:.2f}%')


Inner Fold Accuracy: 45.64%
Inner Fold Accuracy: 57.49%
Inner Fold Accuracy: 48.78%
Inner Fold Accuracy: 50.87%
Inner Fold Accuracy: 47.39%
Inner Fold Accuracy: 50.52%
Inner Fold Accuracy: 49.65%
Inner Fold Accuracy: 54.90%
Inner Fold Accuracy: 46.15%
Inner Fold Accuracy: 47.55%
Outer Fold Mean Inner Accuracy: 49.89%
Outer Fold Accuracy: 49.22%
Inner Fold Accuracy: 49.13%
Inner Fold Accuracy: 48.78%
Inner Fold Accuracy: 52.96%
Inner Fold Accuracy: 46.69%
Inner Fold Accuracy: 52.96%
Inner Fold Accuracy: 54.01%
Inner Fold Accuracy: 54.20%
Inner Fold Accuracy: 54.90%
Inner Fold Accuracy: 47.90%
Inner Fold Accuracy: 52.10%
Outer Fold Mean Inner Accuracy: 51.36%
Outer Fold Accuracy: 52.98%
Inner Fold Accuracy: 50.17%
Inner Fold Accuracy: 52.61%
Inner Fold Accuracy: 52.26%
Inner Fold Accuracy: 50.87%
Inner Fold Accuracy: 49.48%
Inner Fold Accuracy: 53.31%
Inner Fold Accuracy: 45.80%
Inner Fold Accuracy: 48.25%
Inner Fold Accuracy: 48.95%
Inner Fold Accuracy: 50.70%
Outer Fold Mean Inner Accu